# Table of Contents
* [Learning Objectives:](#Learning-Objectives:)
	* [Some Simple Setup](#Some-Simple-Setup)
* [Accessing Array Items](#Accessing-Array-Items)
	* [Indexing](#Indexing)
	* [Slicing](#Slicing)
	* [Important Differences Between Python Slicing and NumPy Slicing](#Important-Differences-Between-Python-Slicing-and-NumPy-Slicing)
	* [Region Selection and Assignment](#Region-Selection-and-Assignment)
	* [Other Common Slicing Patterns](#Other-Common-Slicing-Patterns)
		* [Shifting](#Shifting)
		* [Reversal](#Reversal)
	* [Caveats, Gotchas, and Subtleties](#Caveats,-Gotchas,-and-Subtleties)


# Learning Objectives:

After completion of this module, learners should be able to:

* use and explain *slicing* and *indexing* rules in `numpy`

## Some Simple Setup

We're going to run a few quick commands in IPython to shorten a few names and to make some nice graphics interaction (in this Jupyter notebook).

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import os.path as osp
import numpy.random as npr
vsep = "\n-------------------\n"

def dump_array(arr):
    print("%s array of %s:" % (arr.shape, arr.dtype))
    print(arr)

# Accessing Array Items

## Indexing

Items in NumPy arrays may be accessed using a single index composed of multiple values

![default](img/mef_numpy_selection-noalpha.png)


In [ ]:
arr = np.arange(24).reshape(4,6) # random.randint(11, size=(4, 6))

print("the array:")
print(arr, end=vsep)

print("index [3,2] :", arr[3,2], end=vsep)
print("index [2]   :", arr[2], end=vsep)

# non-idiomatic, creates a view of arr[2] then indexes into that copy
print("index [3][2]:", arr[3][2])

Compare this with indexing into a nested Python list

In [ ]:
aList = [list(row) for row in arr]
print(aList)
print(aList[2][2])

try:
    print(aList[2,2])
except TypeError as e:
    print("Unhappy with multi-value index")
    print("Exception message:", e)

## Slicing

We can also use slicing to select entire row and columns at once:

<center>
![default](img/mef_numpy_slice_01-noalpha.png)
</center>

<center>
![default](img/mef_numpy_slice_02-noalpha.png)
</center>

## Important Differences Between Python Slicing and NumPy Slicing

* Python slicing returns a **copy** of the original data
  * Changing the slice won't change the original.
* NumPy slicing returns a view of the original data
  * Changing the slice **will** change the original data


The [NumPy Indexing Page](http://docs.scipy.org/doc/numpy/reference/arrays.indexing.html) has a lot more information.

In [ ]:
print("array:")
print(arr)

print("\naccessing a row:")
dump_array(arr[2,:])

print("\naccessing a column:")
dump_array(arr[:,2])

print("\na row:", arr[2,:], "has shape:", arr[2,:].shape)
print("\na col:", arr[:,2], "has shape:", arr[:,2].shape)

Bear in mind that numerical indexing will reduce the dimensionality of the array.  Slicing from `index` to `index+1` can be used to keep that dimension if you need it.

In [ ]:
print("lost dimension:", end=' ') 
dump_array(arr[2, 1:4])

print("\nkept dimension:", end=' ') 
dump_array(arr[2:3, 1:4])

## Region Selection and Assignment

Multiple slices, as part of an index, can select a region out of an array

In [ ]:
print("array:")
print(arr)

print("\na sub-array:")
dump_array(arr[1:3, 2:4])

Slices are always views of the underlying array.  Thus, modifying them modifies the underlying array

In [ ]:
arr = np.arange(24).reshape(4,6)
print("even elements (at odd indices) of first row:")
print(arr[0, ::2]) # select every other element from first row

arr[0,::2] = -1   # update is done in-place, no copy

print("\nafter assinging to those:")
print(arr)

In [ ]:
arr = np.arange(24).reshape(4,6)
arr[:, :] = 42
arr

You may have noticed something peculiar in the cell above.  There is an assignment of a scalar to an array.  This is called *broadcasting* and in this simple case, it simply expands the scalar value to fill the elements of the target.  Here's another example:

In [ ]:
arr[0] = 10
print(arr)

And here is one more example of broadcasting and slicing: assigning a value to fill a column.

In [ ]:
arr[:,2] = 99
print(arr)

As with Python lists, empty start/end points in a slice represent the beginning/end of the NumPy array.

In [ ]:
# fill the visual lower-left box with 0
arr[2:,:2] = 0 
print(arr)

In [ ]:
# fill the visual lower-right box with -1
arr[2:,3:] = -1 
print(arr)

We can also assign sequences, if the shapes on the left-hand side and the right-hand side match.

In [ ]:
arr[3,:] = [10, 20, 30, 40, 50, 60]
print(arr)

In [ ]:
arr[::-1, ::-1]

Sequence assignment extends to multi-dimensional objects.

In [ ]:
arr[1:3,3:5] = [[2,4], [8,16]]
print(arr)

In [ ]:
arr.reshape(3,2,4)

In [ ]:
arr.reshape(4,3,2)

## Other Common Slicing Patterns

### Shifting

As with Python, `-index` is the equivalent of `len(seq)-1`.

The `-1` (on the RHS) is an end-point so it is *not* included.  So, the interpretation is that we "do not include the last element". Thus, on the RHS we lose one element and on the LHS we lose one element. Both sides are the same length — the original length of the array minus one — and it is a legal assignment. *But* not all items of `arr` have been updated. Which element is unaffected by the assignment?

In [ ]:
# shift array
arr = 2**np.arange(10)
print("original:")
dump_array(arr)

arr2 = 2**np.arange(10)
arr2[1:] = arr[:-1]
print("\nafter slicing assignment")
dump_array(arr2)

In [ ]:
arr2 - arr

### Reversal

In [ ]:
# using a negative stride indicates walking backward
# it may be surprising, but this is still -not- a copy
# the reverse striding still shares data with the underlying array
rev_arr = arr[::-1]
rev_arr[0] = -99
print(arr)
print(rev_arr)

In [ ]:
arr_copy = arr.copy()
arr[1] = 777
arr, arr_copy

In [ ]:
# Replace the diagonal with a specific value (solution 1)
new = np.arange(36).reshape(6,6)
new[np.diag_indices_from(new)] = 42
new

In [ ]:
# Replace the diagonal with a specific value (solution 2)
new = np.arange(36).reshape(6,6)
new[range(6),range(6)] = 42
new

## Caveats, Gotchas, and Subtleties

* Changing a slice
* Using multiple square brackets instead of a single, comma-separated slice

In [ ]:
arr = np.arange(720).reshape((2,3,4,5,6))
dump_array(arr[0,0,0,0])
dump_array(arr[0][0][0][0])

In [ ]:
dump_array(arr[:,0,0,0])
dump_array(arr[:][0][0][0])
#Big difference! What happened?
#arr has 5 dimensions
#If fewer than 5 dimensions are specificed in the slice, 
# NumPy assumes that the remaining dimensions should be sliced with ":"

In [ ]:
arr[:][0][0][0] == arr[:,:,:,:,:][0][0][0]
#arr[:,:,:,:,:] returns a view of the array that is completely unchanged.
#In other words, arr[:] == arr
print(arr[:].shape)
print(arr[:][:][:].shape)
print(arr[:,0,0])

#So, how is that different from arr[:,0,0,0]?
#The key point here is that NumPy slicing works on multiple axes at a time.
#So, arr[:][0] is logically different from arr[:,0] because
#arr[:][0] is slicing on axis 0, followed by slicing on axis 0 of the new array
#arr[:] returns a view into an array.
#arr[:,0] is slicing on axis 0 and 1.

#arr[0,0,0,0] and arr[0][0][0][0] are only equivalent by special case
#arr[0:2,0,0,0] and arr[0:2][0][0][0] are not equivalent